# Виконав студент группи МСС-3 Калюжний Денис

### **Теорія:**  
Розглянемо проміжок $(-\infty,\infty)$ і вагу $\rho(x)=e^{-x^2}$, тобто виведемо формулу Гаусса для обчислення інтегралу 
<center>$\int\limits_{-\infty}^{\infty} e^{-x^2}f(x)\text{d}x$.</center>  
За теорією 
<center>$w(x) = H_n(x) = (-1)^n e^{x^2} \dfrac{\text{d}^n}{\text{d}x^n} e^{-x^2}$, </center> 
де $H_n(x) -$ багаточлени Ерміта, які обчислюються за рекурентними формулами    
  
    
<center>${H_{n+1}(x) = 2xH_n(x) - 2nH_{n-1}(x)}$,</center>

з початковими умовами $H_{-1}=0, H_0=1$.

Коефіцієнти квадратурної формули обчислюються за формулами
<center>$c_k = \dfrac{2^{n+1}n!\sqrt{n}}{(H'_n(x_k))^2}$,</center>  
а $x_k - $ нулі поліному $H_n(x)$.  

Залишковий член
<center>$R_n(f)=\dfrac{n!\sqrt{\pi}}{2^n(2n)!}f^{2n}(\xi),$</center>
<center>$|R_n(f)|\leq M_{2n} \dfrac{n!\sqrt{\pi}}{2^n(2n)!}.$</center>

### **Задача:**  
Наближено обчислити $I = \int\limits_{-\infty}^{\infty} e^{-x^2} \cos^2(x) \text{d}x$ 
за допомогою формули Гаусса та багаточленів Ерміта.

### **Розв'язок:**  

Імпорт потрібних бібліотек.

In [1]:
import numpy as np
from scipy.optimize import fsolve
from functools import partial

np.set_printoptions(precision=3)

Задамо функцію $f$ та $\rho$.

In [2]:
f = lambda x: np.square(np.cos(x))
r = lambda x: np.exp(-np.square(x))

Визначаємо функцію **Hn**, яка буде рахувати багаточлен Ерміта заданого степеня   
та функцію **dHndx**, що знаходить похідну багаточлена Ерміта.

In [3]:
def Hn(x, n):
    if n == -1:
        hn = 0
    elif n == 0:
        hn = 1
    else:
        hn = 2 * x * Hn(x, n-1) - 2 * (n - 1) * Hn(x, n-2)
    return hn

def dHndx(x, hn):
    n = hn.keywords['n']
    return 2 * n * Hn(x, n-1)

Задамо $n$.

In [4]:
n = 5

Виконуємо обчислення,знаходимо $x_k$, $f(x_k)$.

In [5]:
hn = partial(Hn, n=n)
xk = fsolve(hn, np.array(np.linspace(-3, 3, n)))
# цю частину потрібно контролювати (знаходження коренів багаточлена Ерміта)
try:
    assert len(np.round(xk, 3)) == len(np.unique(np.round(xk, 3))), "duplicate roots"
except AssertionError as e:
    print(e)
else:
    print("xk =", xk)                                

xk = [-2.02  -0.959  0.     0.959  2.02 ]


In [6]:
fxk = f(xk)
print("f(xk) =", fxk)

f(xk) = [0.189 0.33  1.    0.33  0.189]


Шукаємо коефіцієнти $c_k$.

In [7]:
ck = []
for k in range(n):
    factorial = np.prod(np.arange(1, n+1))
    derivate = dHndx(xk[k], hn)
    c = (2**(n+1) * factorial * np.sqrt(np.pi)) / derivate**2
    ck.append(c)
ck = np.array(ck)
print("ck =", ck)

ck = [0.02  0.394 0.945 0.394 0.02 ]


Обчислюємо інтеграл $I_n$.

In [8]:
In = np.sum(ck * fxk)
print("In =", In)

In = 1.212838801839738


Аналітичний розв'язок даної задачі: $I = \dfrac{(1 + \operatorname{e}) \sqrt{\pi}}{2 \operatorname{e}}$ і чисельно він рівний

In [9]:
I = ((1 + np.e) * np.sqrt(np.pi)) / (2 * np.e)
print("I =", I)

I = 1.212251591539404


Отже апостеріорна оцінка похибки $R_n$:

In [10]:
Rn = np.abs(I - In)
print("Rn =", Rn)

Rn = 0.0005872103003339291


Визначимо апріорну оцінку похибки $R$.   
$M_{10} = max\left|\dfrac{\text{d}}{\text{d}x^{10}} \cos^2(x)\right| = max\left| -512 (\cos^2(x) - \sin^2(x)) \right| = 512$

In [11]:
M10 = 512
R = M10 * np.prod(np.arange(1, n+1)) * np.sqrt(np.pi) / (2**n * np.prod(np.arange(1, 2*n + 1)))
print("R =", R)

R = 0.0009378062703203787


Перевіряємо, що оцінки апостеріорна <= апріорна.

In [12]:
try:
    assert Rn <= R, "апостеріорна > апріорна"
except AssertionError as e:
    print(e)
else:
    print('OK')

OK
